In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# Maker Functionality [NB20]


In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=False, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
sdkaddr = SDK.addr()
print(sdkaddr)
SDK.version().get("msg")

0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee


'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(options=(('USDC/ETH', 'USDC/ETH'), ('DAI/USDC', 'DAI/USDC')), value='USDC/ETH')

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [6]:
m = SDK.mGetMarketByPair(pair=pair)
m

MarketByPair(pair='ETH/USDC', bestBid=1989.9999997673926, bestAsk=2005.0000000000175, liqBid=4204.383388, liqAsk=9.720140130459825, lastBid=899.9999999318561, lastAsk=3000.0000000000327)

In [7]:
SDK.roundsd(m.lastBid,6), SDK.roundsd(m.bestBid,6), SDK.roundsd(m.mid,6), SDK.roundsd(m.bestAsk,6), SDK.roundsd(m.lastAsk,6)

(900.0, 1990.0, 1997.5, 2005.0, 3000.0)

In [8]:
(SDK.roundsd(m.liqBid,4), m.liqBidUnit, SDK.roundsd(m.liqAsk,4), m.liqAskUnit)

(4204.0, 'USDC', 9.72, 'ETH')

## Create strategy

#### `mCreateStrategy`

In [9]:
try:
    strat_w()
except:
    strat_w = PcSliderManager(choices=["buyStart", "buyEnd", "sellStart", "sellEnd"], values=[0, 1, 0, 1])
    strat_w()

In [10]:
sellAmountTknB =     1
buyAmountTknQ  =   100
x = strat_w.values
print(f"pair = {pair}, TknB={tknb}, TknQ={tknq}")
print(f"buy  {tknb} {SDK.roundsd(m.ibid(x[0]),6)} down to {SDK.roundsd(m.ibid(x[1]),6)}   [{sellAmountTknB} {tknq}] ")
print(f"sell {tknb} {SDK.roundsd(m.iask(x[2]),6)}      to {SDK.roundsd(m.iask(x[3]),6)}   [{buyAmountTknQ} {tknb}] ")

pair = ETH/USDC, TknB=ETH, TknQ=USDC
buy  ETH 1990.0 down to 900.0   [1 USDC] 
sell ETH 2005.0      to 3000.0   [100 ETH] 


In [11]:
x = strat_w.values
tx = SDK.mCreateStrategy(
    pair, 
    buyRangeStart  = m.ibid(x[0]), buyRangeEnd  = m.ibid(x[1]), buyAmountTknQ = buyAmountTknQ, 
    sellRangeStart = m.iask(x[2]), sellRangeEnd = m.iask(x[3]), sellAmountTknB = sellAmountTknB, 
    overrides = None)
tx

{'data': '0xf727473a000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb480000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000de0b6b3a7640000000000000000000000000000000000000000000000000000000cfeb5cd604e38000000000000000000000000000000000000000000000000000f8ea2d65093c40000000000000000000000000000000000000000000000000000000005f5e1000000000000000000000000000000000000000000000000000000000005f5e10000000000000000000000000000000000000000000000000000000000f51af6d200000000000000000000000000000000000000000000000000000001f75104d4',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x0de0b6b3a7640000'}}

In [12]:
try:
    submit_w()
except:
    submit_w = CheckboxManager(["submit"], values=[0])
    submit_w()

In [22]:
if submit_w.values[0]: 
    r = SDK.signsubmittx(tx, sign=True)
else:
    r = "check box above to submit"
r

{'txReceipt': {'hash': '0x8747099a98a7f218cc8c578bd9c56fd1c097839f33440ea4cbc4544c0e31cb01',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

## Review strategies

In [14]:
data = SDK.mGetUserStrategies(user=sdkaddr)
print("retrieved strategies", len(data))
data[0]

retrieved strategies 2


Strategy(sid=3, baseToken='ETH', quoteToken='USDC', buyPriceLow=1500.0, buyPriceHigh=1600.0, buyBudget=240.26621, sellPriceLow=2500.0, sellPriceHigh=2600.0, sellBudget=1.1650563, encoded=EncodedStrategy(sid=3, order0=EncodedOrder(token='ETH', y=1165056265830556738, z=1165056265830556738, A=2746792519209792, B=4390587127987123), order1=EncodedOrder(token='USDC', y=240266208, z=500000000, A=357520097, B=10901478971)))

In [15]:
d, de=data[0], data[0].encoded
print(de.order1.descr)
print(de.descr[1])
print(d.descr[1])
f"buying {d.baseToken} @", de.order1.p_start, de.order1.p_end, f"{d.quoteToken} per {d.baseToken}"

selling USDC @ (625000000.0473222..666666666.7577751)  [TKNwei] per USDCwei
order1: buying  ETH @ (1.5999999998788553e-09..1.4999999997950062e-09) USDCwei per ETHwei
order1: buying  ETH @ (1600.0..1500.0) USDC per ETH


('buying ETH @',
 1.5999999998788553e-09,
 1.4999999997950062e-09,
 'USDC per ETH')

In [16]:
d, de=data[0], data[0].encoded
print(de.order0.descr)
print(de.descr[0])
print(d.descr[0])
f"selling {d.baseToken} @", 1/de.order0.p_start, 1/de.order0.p_end, f"{d.quoteToken} per {d.baseToken}"

selling ETH @ (2.5e-09..2.6000000000000117e-09)  [TKNwei] per ETHwei
order0: selling ETH @ (2.5e-09..2.6000000000000117e-09) USDCwei per ETHwei
order0: selling ETH @ (2500.0..2600.0) USDC per ETH


('selling ETH @', 2.5e-09, 2.6000000000000117e-09, 'USDC per ETH')

In [17]:
print(f"Number of strategies: {len(data)}")

Number of strategies: 2


In [18]:
# Sim = CarbonSimulatorUI(verbose=False, raiseonerror=True)
# for d in data:
#     Sim.add_fromsdk(d, 6)
# Sim.state()["orders"]

## Delete strategies

In [19]:
stratids = [d["id"] for d in data]
try:
    delete_w()
    assert len(data) == lendata
except:
    delete_w = CheckboxManager(stratids)
    lendata=len(data)
    delete_w()

In [20]:
try:
    submitd_w()
except:
    submitd_w = CheckboxManager(["submit delete"], values=[0])
    submitd_w()

In [21]:
for cid, ds in zip(stratids, delete_w.values):
    if ds:
        data = SDK.deleteStrategy(strategyId=cid)
        #print(data)
        if submitd_w.values[0]:
            result = SDK.signsubmittx(data, sign=True)
            print(f"Deleteting strategy {cid}: ", result)
            delete_w = None
        else:
            print(f"Would delete strategy {cid} if box was ticked ")